#### Импорт необходимых библиотек

In [40]:
import json
import os
import jiwer as jiwer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from vosk import Model, KaldiRecognizer
import wave
import numpy as np
from pydub import AudioSegment
from sklearn.feature_extraction.text import TfidfVectorizer
from fuzzywuzzy import process
from transformers import BertTokenizer, BertModel
import optuna
from gensim.models import Word2Vec
import librosa
import soundfile as sf
import random

#### Константы

In [41]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    # 'hr_bot_clear',
    # 'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    # 'hr_bot_clear.json',
    # 'hr_bot_noise.json',
    'hr_bot_synt.json'
]

VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_VAL_FILE = os.path.join(VAL_DIR, 'luga.json')

In [42]:
# Настройка Vosk модели для распознавания речи
MODEL_PATH = "../model/vosk_model"
model = Model(MODEL_PATH)
print("Модель Vosk загружена успешно.")

Модель Vosk загружена успешно.


In [43]:
# Метки команд
_label = {
    0: "отказ",
    1: "отмена",
    2: "подтверждение",
    3: "начать осаживание",
    4: "осадить на (количество) вагон",
    5: "продолжаем осаживание",
    6: "зарядка тормозной магистрали",
    7: "вышел из межвагонного пространства",
    8: "продолжаем роспуск",
    9: "растянуть автосцепки",
    10: "протянуть на (количество) вагон",
    11: "отцепка",
    12: "назад на башмак",
    13: "захожу в межвагонное пространство",
    14: "остановка",
    15: "вперед на башмак",
    16: "сжать автосцепки",
    17: "назад с башмака",
    18: "тише",
    19: "вперед с башмака",
    20: "прекратить зарядку тормозной магистрали",
    21: "тормозить",
    22: "отпустить",
}

#### Объявление классов

In [44]:
# Датасет для классификации текста
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

In [45]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # output_dim соответствует количеству классов

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Возвращаем тензор с размером [batch_size, output_dim]

##### Объявление функций

In [46]:
# Функция для обработки аудиофайла
def transcribe_audio(audio_file, dir):
    wf = wave.open(f"{dir}/{audio_file}", "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    result_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            result_text += result.get("text", "")

    final_result = json.loads(rec.FinalResult())
    result_text += final_result.get("text", "")

    return result_text

In [47]:
# Функция классификации текста
def classify_text(text, classifier, tokenizer):
    # Преобразование текста в вектор
    text_vector = tokenizer.transform([text]).toarray()
    text_tensor = torch.tensor(text_vector, dtype=torch.float32)

    # Классификация текста
    with torch.no_grad():
        outputs = classifier(text_tensor)

    _, predicted_class = torch.max(outputs, 1)

    return predicted_class.item()

In [49]:
# Аугментация аудио
def augment_audio(file_path):
    # Загрузка аудио файла
    audio, sr = librosa.load(file_path, sr=None)

    # Добавление белого шума
    noise = np.random.randn(len(audio))
    augmented_audio = audio + 0.005 * noise

    # Изменение скорости
    augmented_audio_speed = librosa.effects.time_stretch(audio, rate=1.2)

    # Сохранение аугментированного аудио с добавлением шума
    augmented_file_path = file_path.replace('.wav', '_augmented.wav')
    sf.write(augmented_file_path, augmented_audio, sr)

    # Сохранение аугментированного аудио с изменённой скоростью
    augmented_file_path_speed = file_path.replace('.wav', '_augmented_speed.wav')
    sf.write(augmented_file_path_speed, augmented_audio_speed, sr)

    return [augmented_file_path, augmented_file_path_speed]



In [50]:
# Загрузка аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [51]:
def encode_with_bert(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [52]:
# Преобразование MP3 в WAV
def convert_mp3_to_wav(mp3_filepath):
    try:
        # Убедимся, что pydub правильно обрабатывает MP3
        if not mp3_filepath.endswith('.mp3'):
            print(f"Файл {mp3_filepath} не является MP3.")
            return None

        wav_filepath = mp3_filepath.replace('.mp3', '.wav')
        audio = AudioSegment.from_file(mp3_filepath)  # Используем from_file для универсальности
        audio.export(wav_filepath, format='wav')
        print(f"Файл {mp3_filepath} успешно конвертирован в WAV: {wav_filepath}")
        return wav_filepath
    except Exception as e:
        print(f"Ошибка при конвертации {mp3_filepath} в WAV: {e}")
        return None

In [53]:
def load_dataset():
    audio_files = []
    texts = []
    labels = []

    for annotation_file, data_dir in zip(ANNOTATION_FILES, DATA_DIR_FILES):
        annotation_path = os.path.join(ANNOTATION_DIR, annotation_file)
        print(f"Загрузка аннотаций из: {annotation_path}")
        training_annotations = load_annotations(annotation_path)

        for annotation in training_annotations:
            audio_filepath = os.path.join(DATA_DIR, data_dir, annotation['audio_filepath'])
            # Проверка существования файла перед конвертацией
            if os.path.exists(audio_filepath):
                if audio_filepath.endswith('.mp3'):
                    audio_filepath = convert_mp3_to_wav(audio_filepath)

                augmented_files = augment_audio(audio_filepath)  # Аугментация аудио
                audio_files.extend(augmented_files)

            else:
                print(f"Файл {audio_filepath} не найден.")
                continue  # Пропустить, если файл не найден


            text = annotation['text']
            label = annotation['label']
            texts.extend([text] * len(augmented_files))  # Повторяем текст для каждого аугментированного файла
            labels.extend([label] * len(augmented_files))

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

    print("Загрузка датасета завершена.")
    return audio_files, texts, labels

In [54]:
##### Подготовка настроек модели, создание модели

In [55]:
# Вычисление WER
def calculate_wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    S = sum(1 for r, h in zip(reference_words, hypothesis_words) if r != h)
    D = len(reference_words) - len(hypothesis_words) if len(reference_words) > len(hypothesis_words) else 0
    I = len(hypothesis_words) - len(reference_words) if len(hypothesis_words) > len(reference_words) else 0
    N = len(reference_words)

    return (S + D + I) / N if N > 0 else 0


In [56]:
# Вычисление Mq
def calculate_mq(wer, f1_weighted):
    WERnorm = wer
    return 0.25 * (1 - WERnorm) + 0.75 * f1_weighted

In [61]:
# В функции objective:
def objective(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 4, 16)

    # Загрузка и подготовка данных
    audio_files, texts, labels = load_dataset()
    train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

    # Генерация векторов BERT
    bert_vectors = np.array([encode_with_bert(text).detach().numpy() for text in train_texts])

    # Преобразование меток в тензор
    train_labels = torch.tensor(train_labels, dtype=torch.long)  # Метки должны быть типа long

    # Создание датасета и загрузчика
    train_dataset = TextDataset(bert_vectors, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Определение количества классов (например, 21)
    num_classes = len(set(labels))

    # Обучение модели
    classifier = TextClassifier(input_dim=768, hidden_dim=hidden_dim, output_dim=num_classes)  # Количество классов
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

    for epoch in range(3):  # Можно увеличить количество эпох
        for texts_batch, labels_batch in train_loader:
            texts_batch = texts_batch.float()
            optimizer.zero_grad()
            outputs = classifier(texts_batch)

            # Если метки представлены как one-hot векторы, преобразуем их в индексы
            if labels_batch.ndim > 1:
                labels_batch = torch.argmax(labels_batch, dim=1)  # Преобразование в индексы классов

            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()

    # Оценка модели на тестовом наборе
    test_bert_vectors = np.array([encode_with_bert(text).detach().numpy() for text in test_texts])
    test_dataset = TextDataset(test_bert_vectors, test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for texts_batch, labels_batch in test_loader:
            texts_batch = texts_batch.float()
            outputs = classifier(texts_batch)

            # Получаем предсказанные классы
            _, predicted = torch.max(outputs, 1)

            all_predictions.extend(predicted.numpy())
            all_labels.extend(labels_batch.numpy())

    # Вычисление метрик
    f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')

    # Расчет WER и Mq
    wer = calculate_wer(" ".join(test_texts), " ".join([tokenizer.decode(pred) for pred in all_predictions]))
    mq = calculate_mq(wer, f1_weighted)

    print(f'F1-Weighted: {f1_weighted}, Precision: {precision}, Recall: {recall}, WER: {wer}, Mq: {mq}')

    return f1_weighted

In [62]:
# Использование BERT для контекстуальной векторизации
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [63]:
# Запуск оптимизации гиперпараметров
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print("Лучшие гиперпараметры: ", study.best_params)

[I 2024-10-13 01:27:04,903] A new study created in memory with name: no-name-c279418f-fa14-427c-a88a-9f12c2c8da9c
C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2379490164.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Загрузка аннотаций из: ../data/train/annotation/hr_bot_synt.json
Загрузка датасета завершена.


[W 2024-10-13 01:37:30,144] Trial 0 failed with parameters: {'hidden_dim': 75, 'learning_rate': 6.945649213089256e-05, 'batch_size': 16} because of the following error: RuntimeError('Expected target size [16, 21], got [16]').
Traceback (most recent call last):
  File "D:\russian_railways\002_train_operator_console\venv\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\nikit\AppData\Local\Temp\ipykernel_30368\2379490164.py", line 39, in objective
    loss = criterion(outputs, labels_batch)
  File "D:\russian_railways\002_train_operator_console\venv\lib\site-packages\torch\nn\modules\module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "D:\russian_railways\002_train_operator_console\venv\lib\site-packages\torch\nn\modules\module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "D:\russian_railways\002_train_operator_console\venv\lib\site-packages\

RuntimeError: Expected target size [16, 21], got [16]